In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
tickers = ["AXP","AAPL","BA","CAT","CVX","CSCO","DIS","DOW", "XOM",
           "HD","IBM","INTC","JNJ","KO","MCD","MMM","MRK","MSFT",
           "NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT","WBA"]


In [3]:
#list of tickers whose financial data needs to be extracted
financial_dir = {}

for ticker in tickers:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c)"}) # try soup.findAll("table") if this line gives error 
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        
        #combining all extracted information with the corresponding ticker
        financial_dir[ticker] = temp_dir
    except:
        print("Problem scraping data for ",ticker)

In [4]:
#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]

In [5]:
combined_financials

,AXP,AAPL,BA,CAT,CVX,CSCO,DIS,DOW,XOM,HD,...,NKE,PFE,PG,TRV,UTX,UNH,VZ,V,WMT,WBA
% held by insiders,0.10%,0.07%,0.13%,0.19%,0.05%,0.04%,0.20%,0.08%,0.08%,0.12%,...,1.17%,0.04%,0.11%,0.29%,N/A,0.71%,0.03%,0.20%,50.91%,22.25%
% held by institutions,87.09%,62.12%,66.60%,70.02%,68.96%,73.98%,66.29%,70.82%,56.77%,72.20%,...,85.83%,72.56%,65.03%,84.50%,N/A,89.92%,68.43%,98.06%,30.95%,60.61%
200-day moving average,95.07,86.42,169.77,124.08,86.74,43.19,114.53,38.50,43.50,238.40,...,94.36,35.65,121.28,109.84,103.75,287.80,56.39,186.38,124.79,42.43
5-year average dividend yield,1.53,1.54,2.58,3.01,4.22,2.99,1.44,N/A,4.40,2.10,...,1.07,3.59,3.01,2.30,N/A,1.45,4.45,0.63,2.37,2.48
50-day moving average,99.00,113.87,170.58,140.68,86.27,44.08,127.47,44.72,41.77,276.44,...,106.21,37.94,134.70,115.72,57.76,311.56,58.67,201.47,134.16,39.66
52-week change,-13.78%,107.52%,-58.48%,12.02%,-35.68%,-20.91%,-3.48%,1.04%,-49.06%,16.55%,...,31.46%,-3.41%,11.94%,-23.59%,N/A,30.39%,-0.72%,13.69%,16.50%,-38.08%
52-week high,138.13,137.98,391.00,155.48,125.27,50.28,153.41,56.25,75.18,292.95,...,119.24,40.97,141.70,149.58,158.44,324.57,62.22,217.35,151.33,64.50
52-week low,67.00,53.15,89.00,87.50,51.60,32.40,79.07,21.95,30.11,140.63,...,60.00,27.88,94.34,76.99,48.05,187.72,48.84,133.93,102.00,33.88
Accounts payable,"-92,25,000","70,94,000","-35,83,000","-18,14,000","-52,05,000","1,41,000","1,37,78,000","-13,94,000","2,46,94,000","36,44,000",...,"22,48,000",-,"1,20,71,000",NaN,"1,08,09,000","67,78,000","77,25,000","5,000","20,20,000","82,000"
Accounts receivable,NaN,"-36,19,000","1,58,000",NaN,NaN,"-1,07,000",NaN,NaN,NaN,NaN,...,"-4,32,000",-,"6,34,000","-1,10,000","8,46,000","-24,73,000",-,"94,000",NaN,"82,000"


In [6]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & amortisation",
         "Market cap (intra-day)",
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt",
         "Forward annual dividend yield"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]

In [7]:
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)

In [9]:
# cleansing of fundamental data imported in dataframe
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)
tickers = all_stats_df.columns

In [10]:
all_stats_df

,AAPL,CAT,CSCO,DOW,HD,IBM,INTC,JNJ,KO,MCD,...,MRK,MSFT,NKE,PFE,PG,UNH,VZ,V,WMT,WBA
EBITDA,7.867000e+07,8.580000e+06,1.582000e+07,5.680000e+06,1.910000e+07,1.558000e+07,3.812000e+07,2.721000e+07,1.125000e+07,8.910000e+06,...,1.905000e+07,6.526000e+07,4.230000e+06,2.005000e+07,1.919000e+07,2.649000e+07,4.751000e+07,1.548000e+07,3.662000e+07,5.020000e+06
D&A,1.153300e+07,2.511000e+06,1.808000e+06,2.876000e+06,2.411000e+06,6.632000e+06,1.176000e+07,7.016000e+06,1.511000e+06,1.674000e+06,...,3.650000e+06,1.279600e+07,7.140000e+05,5.406000e+06,3.013000e+06,2.867000e+06,1.655000e+07,7.430000e+05,1.111300e+07,1.973000e+06
MarketCap,1.910000e+09,8.340000e+07,1.688800e+08,3.706000e+07,2.981500e+08,1.084300e+08,2.100100e+08,3.903900e+08,2.197100e+08,1.627400e+08,...,2.142000e+08,1.540000e+09,1.845400e+08,2.047300e+08,3.445800e+08,2.861600e+08,2.474200e+08,4.410300e+08,3.872900e+08,3.020000e+07
NetIncome,5.842400e+07,4.142000e+06,1.121400e+07,-1.976000e+06,1.182700e+07,7.878000e+06,2.366100e+07,1.518500e+07,9.189000e+06,4.770800e+06,...,1.047900e+07,4.428100e+07,4.304000e+06,1.417100e+07,1.276400e+07,1.709800e+07,1.914500e+07,1.175400e+07,1.789500e+07,7.600000e+05
CashFlowOps,8.000800e+07,5.724000e+06,1.542600e+07,6.503000e+06,2.000900e+07,1.512200e+07,3.791400e+07,2.073500e+07,8.756000e+06,5.508900e+06,...,1.309400e+07,6.067500e+07,4.996000e+06,1.496700e+07,1.740300e+07,2.230100e+07,4.346200e+07,1.238600e+07,3.302600e+07,5.777000e+06
Capex,-8.302000e+06,-2.442000e+06,-7.700000e+05,-1.717000e+06,-2.464000e+06,-2.996000e+06,-1.601400e+07,-3.292000e+06,-1.823000e+06,-2.068100e+06,...,-3.750000e+06,-1.544100e+07,-1.060000e+06,-2.363000e+06,-3.073000e+06,-2.014000e+06,-2.232200e+07,-8.170000e+05,-9.403000e+06,-1.418000e+06
CurrAsset,1.628190e+08,3.919300e+07,4.357300e+07,1.681500e+07,1.981000e+07,3.842000e+07,3.123900e+07,4.527400e+07,2.041100e+07,3.557900e+06,...,2.748300e+07,1.819150e+08,2.055600e+07,3.280300e+07,2.798700e+07,4.263400e+07,3.747300e+07,2.097000e+07,6.180600e+07,1.870000e+07
CurrLiab,1.057180e+08,2.662100e+07,2.533100e+07,1.067900e+07,1.837500e+07,3.770100e+07,2.231000e+07,3.596400e+07,2.697300e+07,3.621000e+06,...,2.222000e+07,7.231000e+07,8.284000e+06,3.730400e+07,3.297600e+07,6.178200e+07,4.486800e+07,1.341500e+07,7.779000e+07,2.576900e+07
PPE,3.737800e+07,1.290400e+07,2.453000e+06,2.306800e+07,2.836500e+07,1.500500e+07,5.538600e+07,1.765800e+07,1.083800e+07,3.742120e+07,...,1.505300e+07,5.290400e+07,7.963000e+06,1.396700e+07,2.069200e+07,8.704000e+06,1.146090e+08,2.695000e+06,1.270490e+08,1.347900e+07
BookValue,9.048800e+07,1.458800e+07,3.792000e+07,1.354100e+07,-3.116000e+06,2.084100e+07,7.765900e+07,5.947100e+07,1.898100e+07,-8.210300e+06,...,2.590700e+07,1.183040e+08,8.055000e+06,6.314300e+07,4.652100e+07,5.761600e+07,6.139500e+07,3.468400e+07,7.466900e+07,2.351200e+07


In [11]:
# calculating relevant financial metrics for each stock
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]

In [12]:
final_stats_df

,EBIT,TEV,EarningYield,FCFYield,ROC,BookToMkt,DivYield
AAPL,67137000.0,1.944706e+09,0.034523,0.046236,0.710602,0.047376,0.0072
CAT,6069000.0,9.698100e+07,0.062579,0.097914,0.238224,0.174916,0.0275
CSCO,14012000.0,1.622160e+08,0.086379,0.095902,0.677072,0.224538,0.0364
DOW,2804000.0,4.689900e+07,0.059788,0.221802,0.096014,0.365380,0.0569
HD,16689000.0,3.253850e+08,0.051290,0.075375,0.560034,-0.010451,0.0217
IBM,8948000.0,1.618130e+08,0.055298,0.167094,0.569066,0.192207,0.0533
INTC,26360000.0,2.263890e+08,0.116437,0.256788,0.409858,0.369787,0.0270
JNJ,20194000.0,4.075740e+08,0.049547,0.061546,0.748813,0.152337,0.0270
KO,9739000.0,2.537880e+08,0.038375,0.048150,2.277596,0.086391,0.0328
MCD,7236000.0,1.969212e+08,0.036746,0.046559,0.193693,-0.050450,0.0232


In [13]:
# finding value stocks based on Magic Formula
final_stats_val_df = final_stats_df.loc[tickers,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)

------------------------------------------------
Value stocks based on Greenblatt's Magic Formula
      EarningYield       ROC  MagicFormulaRank
MRK       0.066473  0.758023               1.0
CSCO      0.086379  0.677072               2.0
PFE       0.059726  1.547010               3.0
INTC      0.116437  0.409858               4.0
KO        0.038375  2.277596               5.0
VZ        0.087273  0.288768               6.0
WBA       0.062997  0.475351               7.0
MMM       0.060974  0.478910               8.0
PG        0.043358  1.030185               9.0
JNJ       0.049547  0.748813              10.0
IBM       0.055298  0.569066              11.0
CAT       0.062579  0.238224              12.0
HD        0.051290  0.560034              13.0
V         0.032734  1.437756              14.0
UNH       0.069050 -2.261873              15.0
AAPL      0.034523  0.710602              16.0
WMT       0.057064  0.229658              17.0
DOW       0.059788  0.096014              18.0
MSFT     

In [14]:
# finding highest dividend yield stocks
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)

------------------------------------------------
Highest dividend paying stocks
DOW     0.0569
WBA     0.0539
IBM     0.0533
PFE     0.0426
VZ      0.0422
CSCO    0.0364
MMM     0.0360
KO      0.0328
MRK     0.0292
CAT     0.0275
JNJ     0.0270
INTC    0.0270
MCD     0.0232
PG      0.0231
HD      0.0217
UNH     0.0164
WMT     0.0158
MSFT    0.0099
NKE     0.0085
AAPL    0.0072
V       0.0060
Name: DivYield, dtype: float64


In [15]:
# # Magic Formula & Dividend yield combined
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)

------------------------------------------------
Magic Formula and Dividend Yield combined
      EarningYield       ROC  DivYield  CombinedRank
PFE       0.059726  1.547010    0.0426           1.0
CSCO      0.086379  0.677072    0.0364           2.0
MRK       0.066473  0.758023    0.0292           3.0
WBA       0.062997  0.475351    0.0539           4.0
VZ        0.087273  0.288768    0.0422           5.0
IBM       0.055298  0.569066    0.0533           6.0
INTC      0.116437  0.409858    0.0270           7.0
KO        0.038375  2.277596    0.0328           8.0
MMM       0.060974  0.478910    0.0360           9.0
DOW       0.059788  0.096014    0.0569          10.0
JNJ       0.049547  0.748813    0.0270          11.0
CAT       0.062579  0.238224    0.0275          12.0
PG        0.043358  1.030185    0.0231          13.0
HD        0.051290  0.560034    0.0217          14.0
UNH       0.069050 -2.261873    0.0164          15.0
V         0.032734  1.437756    0.0060          16.0
WMT     